# Automating websearch for the query term

Using the python file `screenshot_automation.py` for automated screenshot of the query results in Chrome for query term: "Childhood cancer early diagnosis methods". selenium.webdriver is used inorder to automate the query and the final screenshot if saved as screenshot.png.

In [2]:
%run screenshot_automation.py

 Now, the obtained screenshot is used to scrape info using pytesseract library 

In [3]:
pip install tesseract


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install mysql-connector-python


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import pytesseract
import matplotlib.pyplot as plt
import os
import re
import pandas as pd
import mysql.connector

In [6]:
# specify folder where photos are
pic_folder = 'Web_Photos'

# list all files in the folder
pic_names = os.listdir(pic_folder)

# specify folder for text
text_folder = 'Extracted_Text'

## using image files to extract text

In [7]:
# need tesseract as environment variable in path
for pic_name in pic_names:
    image_path = os.path.join(pic_folder, pic_name)
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    _, thresh_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    text = pytesseract.image_to_string(thresh_image)
    txt_path = os.path.join(text_folder, pic_name.split('.')[0] + '.txt')
    
    with open(txt_path, 'w') as text_file:
        text_file.write(text)
    

In [8]:
# list all text files in the folder
text_names = os.listdir(text_folder)
print(text_names)

['yahoo.txt', 'google.txt', 'bing.txt', 'duckgo.txt', 'duckduckgo.txt']


In [9]:
# regex pattern: starts with h, however many letters in between, ":/", however many letters in between up until there is a space
url_dict = {}

for text_name in text_names:
    txt_path = os.path.join(text_folder, text_name.split('.')[0] + '.txt')
    
    with open(txt_path, 'r') as text_file:
        file_contents = text_file.read()

        if 'yahoo' in text_name:
            urls = re.findall(r'www.[a-zA-Z]*[^\s]*', file_contents)

        else:
            urls = re.findall(r'h[a-zA-Z]*:/[^\s]*', file_contents)
        #print(urls)

    url_dict[text_name] = urls

In [10]:

for key, value in url_dict.items():
    print(key, value)
    print("\n")

yahoo.txt ['www.iccp-portal.org', 'www.paho.org', 'www.ncbi.nim.nih.gov', 'www.nature.com', 'www.thelancet.com', 'www.cancer.org']


google.txt ['https://together.stjude.org', 'https:/Awww.cancer.org', 'https:/Awww.cancer.net', 'https:/Awww.ncbi.nim.nih.gov', 'https://iris.paho.org', 'https:/Awww.ncbi.nim.nih.gov', 'https:/Awww.cancercenter.com', 'https:/Awww.acco.org', 'https://my.clevelandclinic.org', 'https://Awww.paho.org', 'https:/Awww.advocatechildrenshospital.com', 'https://Awww.who.int', 'https://obmjopen.bmj.com', 'https:/Awww.cancer.org', 'https:/Awww.yalemedicine.org', 'https://bmchealthservres.biomedcentral.com', 'https:/Awww.cancer.gov', 'https:/Awww.who.int/cancer']


bing.txt ['https:/Awww.nyp.org/pediatrics', 'https:/Avww.texaschildrens.org/cancer/treatment', 'https:/Avww.mskcc.org/cancer-care/experts', 'https:/Avww.azioinpractice.com', 'https:/Avww.ncbi.nim.nih.gov/pme/articles/PMC8651632', 'https:/Awww.iccp-portal.org/resources/early...', 'https:/Avww.aarp.org/cancern

In [11]:
for k, v in url_dict.items():
    for i in range(len(v)):
        v[i] = re.sub(r'^h[a-z]*:', 'https:', v[i])
        v[i] = re.sub(r':/A\w\w\w\.', '://www.', v[i])


for key, value in url_dict.items():
    print(key, value)
    print("\n")

yahoo.txt ['www.iccp-portal.org', 'www.paho.org', 'www.ncbi.nim.nih.gov', 'www.nature.com', 'www.thelancet.com', 'www.cancer.org']


google.txt ['https://together.stjude.org', 'https://www.cancer.org', 'https://www.cancer.net', 'https://www.ncbi.nim.nih.gov', 'https://iris.paho.org', 'https://www.ncbi.nim.nih.gov', 'https://www.cancercenter.com', 'https://www.acco.org', 'https://my.clevelandclinic.org', 'https://Awww.paho.org', 'https://www.advocatechildrenshospital.com', 'https://Awww.who.int', 'https://obmjopen.bmj.com', 'https://www.cancer.org', 'https://www.yalemedicine.org', 'https://bmchealthservres.biomedcentral.com', 'https://www.cancer.gov', 'https://www.who.int/cancer']


bing.txt ['https://www.nyp.org/pediatrics', 'https://www.texaschildrens.org/cancer/treatment', 'https://www.mskcc.org/cancer-care/experts', 'https://www.azioinpractice.com', 'https://www.ncbi.nim.nih.gov/pme/articles/PMC8651632', 'https://www.iccp-portal.org/resources/early...', 'https://www.aarp.org/cancern

Add the frequency search term count to dataframe.

In [12]:
# add dictionary of urls into data frame
from inserting_urls_into_dataframe import separate_url

In [13]:
df_search = separate_url(url_dict)

In [14]:
df_search.head()

,search_engine,urls
0,yahoo.txt,www.iccp-portal.org
1,yahoo.txt,www.paho.org
2,yahoo.txt,www.ncbi.nim.nih.gov
3,yahoo.txt,www.nature.com
4,yahoo.txt,www.thelancet.com


In [15]:
df_search.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   search_engine  60 non-null     object
 1   urls           60 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


In [16]:
# cleaning the search_engine names
df_search['search_engine'] = df_search['search_engine'].str.replace('.txt','')

In [17]:
# Converting the datatype of urls to string
#df_search['urls'] = df_search['urls'].astype(str)
df_search.head()

,search_engine,urls
0,yahoo,www.iccp-portal.org
1,yahoo,www.paho.org
2,yahoo,www.ncbi.nim.nih.gov
3,yahoo,www.nature.com
4,yahoo,www.thelancet.com


Count the frequency of each search term in the URL

In [18]:
from getting_count_of_searchterms import term_counts
df_search = term_counts(df_search)


In [27]:
df_search.head()

,search_engine,urls,term_count_Childhood,term_count_cancer,term_count_early,term_count_diagnosis,term_count_methods
0,yahoo,www.iccp-portal.org,0,0,0,0,0
1,yahoo,www.paho.org,0,0,0,0,0
2,yahoo,www.ncbi.nim.nih.gov,0,0,0,0,0
3,yahoo,www.nature.com,0,0,0,0,0
4,yahoo,www.thelancet.com,0,0,0,0,0


Connecting to the Mysql database

In [24]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user=input("Enter your database username: "),
  password=input("Enter your database password: "),
  database = "MY_CUSTOM_BOT"
)

print(mydb)

In [31]:
mycursor = mydb.cursor()

mycursor.execute("CREATE TABLE search_data (search_engine VARCHAR(255), urls VARCHAR(255),term_count_Childhood INT ,term_count_cancer INT ,term_count_early INT ,term_count_diagnosis INT ,term_count_methods INT )")

